# Red neuronal recurrente para toxicidad (GRU vs BiGRU)

En este notebook entrenamos dos modelos secuenciales:

1. **GRU**
2. **Bidirectional GRU (BiGRU)**

Partimos del dataset limpio:

- Texto: `text_classic`
- Etiqueta objetivo: `IsToxic`

Incluimos:

- Tokenización con Keras
- Padding
- Entrenamiento con validación
- Métricas en train/test
- Control de overfitting (< 5%)
- Selección del mejor modelo
- Guardado de:
  - Modelo neuronal en `.keras`
  - Métricas en `.json` (formato estándar del equipo)

Este JSON se usa directamente en `comparison_models.ipynb`.


### 1. Importación de librerías y configuración

En esta celda:

- Importamos:
  - `pandas`, `numpy`
  - `tensorflow/keras` para la red neuronal
  - métricas de `sklearn` para mantener el mismo estándar del equipo
- Definimos:
  - semilla
  - nombre base del modelo
  - parámetros de tokenización


In [ ]:
# === 1. Imports and config ===================================================

import json  # To save metrics in JSON format
from datetime import datetime  # To generate ISO timestamps
from pathlib import Path  # To handle project paths

import numpy as np  # Numerical operations
import pandas as pd  # DataFrame handling

import tensorflow as tf  # Deep learning framework
from tensorflow.keras import layers, models  # Keras high-level API

from sklearn.model_selection import train_test_split  # Data split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)

print("✅ Librerías importadas correctamente")

RANDOM_STATE = 42
TARGET_COL = "IsToxic"

T# Text column selection for RNN
# We default to text_classic for consistency with the cleaned dataset.
# You can switch to text_basic to test a less normalized variant.
TEXT_COL = "text_classic"  # or "text_basic"

model_name_base = "bigru_IsToxic"

# Tokenization parameters
MAX_WORDS = 20000
MAX_LEN = 200

tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)


### 2. Funciones auxiliares comunes

Incluimos:

- detección de raíz del proyecto
- creación de carpetas
- cálculo de métricas estándar
- control de overfitting
- guardado del JSON

Nota:
- aquí **no guardamos `.pkl`** porque en redes neuronales
  el artefacto correcto es `.keras`.


In [ ]:
# === 2. Helper functions (NN version) =======================================

def get_project_root() -> Path:
    """Return PX_NLP_G4 project root folder, robust from notebooks."""
    cwd = Path.cwd()
    parts = cwd.parts
    if "notebooks" in parts:
        return cwd.parents[1]
    if cwd.name == "backend":
        return cwd.parent
    return cwd


def build_project_dirs(project_root: Path) -> dict:
    """Create and return standard directories used across notebooks."""
    data_dir = project_root / "data"
    preproc_dir = data_dir / "preprocessing_data"
    results_dir = data_dir / "results"
    models_dir = project_root / "backend" / "models"

    preproc_dir.mkdir(parents=True, exist_ok=True)
    results_dir.mkdir(parents=True, exist_ok=True)
    models_dir.mkdir(parents=True, exist_ok=True)

    return {
        "data_dir": data_dir,
        "preproc_dir": preproc_dir,
        "results_dir": results_dir,
        "models_dir": models_dir
    }


def compute_binary_metrics(y_true, y_pred, y_proba=None) -> dict:
    """Compute standard binary metrics for JSON consistency."""
    out = {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "precision": float(precision_score(y_true, y_pred, zero_division=0)),
        "recall": float(recall_score(y_true, y_pred, zero_division=0)),
        "f1": float(f1_score(y_true, y_pred, zero_division=0)),
    }
    if y_proba is not None:
        try:
            out["roc_auc"] = float(roc_auc_score(y_true, y_proba))
        except Exception:
            out["roc_auc"] = None
    else:
        out["roc_auc"] = None
    return out


def compute_confusion_dict(y_true, y_pred) -> dict:
    """Return confusion matrix values tn, fp, fn, tp."""
    cm = confusion_matrix(y_true, y_pred)
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
    else:
        tn = fp = fn = tp = 0
    return {"tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)}


def overfitting_check(train_metrics: dict, test_metrics: dict, threshold: float = 0.05) -> dict:
    """Flag overfitting risk if train - test > threshold."""
    report = {"threshold": threshold, "flags": {}}
    for k in ["accuracy", "precision", "recall", "f1", "roc_auc"]:
        tr = train_metrics.get(k)
        te = test_metrics.get(k)
        if tr is None or te is None:
            report["flags"][k] = None
            continue
        diff = float(tr - te)
        report["flags"][k] = {
            "train": float(tr),
            "test": float(te),
            "diff": float(diff),
            "overfit_risk": bool(diff > threshold)
        }
    return report


def build_results_json_nn(
    model_name: str,
    target_label: str,
    n_samples: int,
    vocab_size: int,
    train_size: float,
    test_size: float,
    random_state: int,
    test_metrics: dict,
    cm_dict: dict,
    notes: str = ""
) -> dict:
    """Standard team JSON for NN models."""
    return {
        "model_name": model_name,
        "task": "binary_classification",
        "target_label": target_label,
        "data": {
            "n_samples": int(n_samples),
            "n_features_text": int(vocab_size),
            "n_features_numeric": 0,
            "train_size": float(train_size),
            "test_size": float(test_size),
            "random_state": int(random_state),
        },
        "metrics": {
            "accuracy": test_metrics.get("accuracy"),
            "precision": test_metrics.get("precision"),
            "recall": test_metrics.get("recall"),
            "f1": test_metrics.get("f1"),
            "roc_auc": test_metrics.get("roc_auc"),
        },
        "confusion_matrix": cm_dict,
        "timestamp": datetime.now().isoformat(timespec="seconds"),
        "notes": notes
    }


def save_results_json(results: dict, results_dir: Path, model_name: str) -> Path:
    """Save JSON results using model_name as filename."""
    json_path = results_dir / f"{model_name}.json"
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    return json_path


### 3. Carga del dataset limpio y split

En esta celda:

- Cargamos el CSV limpio.
- Verificamos columnas.
- Dividimos en train/test.


In [ ]:
# === 3. Load dataset and split ==============================================

from IPython.display import display, Markdown

project_root = get_project_root()
dirs = build_project_dirs(project_root)

preproc_dir = dirs["preproc_dir"]
results_dir = dirs["results_dir"]
models_dir = dirs["models_dir"]

csv_path = preproc_dir / "youtoxic_english_1000_clean.csv"

df = pd.read_csv(csv_path)

if TEXT_COL not in df.columns:
    raise ValueError(f"La columna seleccionada '{TEXT_COL}' no existe en el dataset limpio.")

missing_cols = [c for c in [TEXT_COL, TARGET_COL] if c not in df.columns]
if missing_cols:
    raise ValueError(f"Faltan columnas en el dataset limpio: {missing_cols}")

X = df[TEXT_COL].astype(str).values
y = df[TARGET_COL].astype(int).values

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y
)

display(Markdown(f"""
**✅ Dataset cargado y split completado**

- Train: **{len(X_train)}**
- Test: **{len(X_test)}**
"""))


### 4. Tokenización y padding

En esta celda:

- Creamos un `Tokenizer`.
- Convertimos textos a secuencias.
- Aplicamos padding.

Esto convierte texto a formato usable por GRU/BiGRU.


In [ ]:
# === 4. Tokenize and pad sequences ==========================================

tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=MAX_WORDS,
    oov_token="<OOV>"
)

tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(
    X_train_seq, maxlen=MAX_LEN, padding="post", truncating="post"
)
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(
    X_test_seq, maxlen=MAX_LEN, padding="post", truncating="post"
)

vocab_size = min(MAX_WORDS, len(tokenizer.word_index) + 1)

display(Markdown(f"""
**✅ Tokenización completada**

- Tamaño vocabulario usado: **{vocab_size}**
- Longitud máxima secuencia: **{MAX_LEN}**
"""))


### 5. Definición de modelos GRU y BiGRU

En esta celda:

- Definimos dos arquitecturas simples y comparables.
- Usamos:
  - Embedding
  - GRU (normal)
  - GRU bidireccional

Buscamos equilibrio entre rendimiento y evitar overfitting.


In [ ]:
# === 5. Build GRU and BiGRU models ==========================================

def build_gru_model(vocab_size: int) -> tf.keras.Model:
    """Build a simple GRU binary classifier."""
    model = models.Sequential([
        layers.Embedding(vocab_size, 128, input_length=MAX_LEN),
        layers.GRU(64),
        layers.Dropout(0.3),
        layers.Dense(1, activation="sigmoid")
    ])
    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model


def build_bigru_model(vocab_size: int) -> tf.keras.Model:
    """Build a Bidirectional GRU binary classifier."""
    model = models.Sequential([
        layers.Embedding(vocab_size, 128, input_length=MAX_LEN),
        layers.Bidirectional(layers.GRU(64)),
        layers.Dropout(0.3),
        layers.Dense(1, activation="sigmoid")
    ])
    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

gru_model = build_gru_model(vocab_size)
bigru_model = build_bigru_model(vocab_size)

display(Markdown("**🧠 Modelos GRU y BiGRU definidos y compilados**"))


### 6. Entrenamiento de GRU

Entrenamos con validación interna.

Usamos `EarlyStopping` para:

- reducir overfitting
- cumplir el requisito de la rúbrica


In [ ]:
# === 6. Train GRU ============================================================

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=2,
    restore_best_weights=True
)

display(Markdown("### 🏋️ Entrenamiento GRU"))

history_gru = gru_model.fit(
    X_train_pad, y_train,
    validation_split=0.2,
    epochs=8,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

print("✅ Entrenamiento GRU completado")


### 7. Entrenamiento de BiGRU


In [ ]:
# === 7. Train BiGRU ==========================================================

display(Markdown("### 🏋️ Entrenamiento BiGRU"))

history_bigru = bigru_model.fit(
    X_train_pad, y_train,
    validation_split=0.2,
    epochs=8,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

print("✅ Entrenamiento BiGRU completado")


### 7.1 Curvas de entrenamiento

En esta celda:

- Visualizamos la evolución de:
  - `loss`
  - `accuracy`

para GRU y BiGRU.

Esto nos ayuda a detectar señales tempranas de overfitting.


In [ ]:
# === 7.1 Training curves =====================================================

from IPython.display import display, Markdown
import matplotlib.pyplot as plt

display(Markdown("### 📉 Curvas de entrenamiento — GRU vs BiGRU"))

def plot_history(history, title):
    """Plot training/validation loss and accuracy for a Keras model."""
    hist = history.history

    # Loss
    fig, ax = plt.subplots(figsize=(6.5, 4))
    ax.plot(hist.get("loss", []), label="train_loss")
    ax.plot(hist.get("val_loss", []), label="val_loss")
    ax.set_title(f"Loss — {title}")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Loss")
    ax.legend()
    plt.tight_layout()
    plt.show()

    # Accuracy
    fig, ax = plt.subplots(figsize=(6.5, 4))
    ax.plot(hist.get("accuracy", []), label="train_acc")
    ax.plot(hist.get("val_accuracy", []), label="val_acc")
    ax.set_title(f"Accuracy — {title}")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Accuracy")
    ax.legend()
    plt.tight_layout()
    plt.show()


plot_history(history_gru, "GRU")
plot_history(history_bigru, "BiGRU")

display(Markdown("""
**🔎 Interpretación**

- Si `val_loss` sube mientras `train_loss` baja,
  es una señal típica de overfitting.
- `EarlyStopping` suele mitigar este efecto en datasets pequeños.
"""))


### 8. Evaluación en train/test + control de overfitting

En esta celda:

- Generamos predicciones y probabilidades
- Calculamos métricas estándar
- Comparamos train vs test


In [ ]:
# === 8. Evaluate GRU and BiGRU ==============================================

def eval_keras_model(model, X_tr, y_tr, X_te, y_te):
    """Compute train/test metrics for a Keras binary classifier."""
    # Train predictions
    tr_proba = model.predict(X_tr, verbose=0).ravel()
    tr_pred = (tr_proba >= 0.5).astype(int)
    train_metrics = compute_binary_metrics(y_tr, tr_pred, tr_proba)

    # Test predictions
    te_proba = model.predict(X_te, verbose=0).ravel()
    te_pred = (te_proba >= 0.5).astype(int)
    test_metrics = compute_binary_metrics(y_te, te_pred, te_proba)

    cm_dict = compute_confusion_dict(y_te, te_pred)
    overfit = overfitting_check(train_metrics, test_metrics, threshold=0.05)

    return train_metrics, test_metrics, cm_dict, overfit


display(Markdown("### 📊 Evaluación GRU"))
train_m_gru, test_m_gru, cm_gru, overfit_gru = eval_keras_model(
    gru_model, X_train_pad, y_train, X_test_pad, y_test
)

display(Markdown(f"""
**GRU — Test**
- Accuracy: `{test_m_gru['accuracy']:.3f}`
- Precision: `{test_m_gru['precision']:.3f}`
- Recall: `{test_m_gru['recall']:.3f}`
- F1: `{test_m_gru['f1']:.3f}`
- ROC-AUC: `{test_m_gru['roc_auc']:.3f}`
"""))

display(Markdown("### 📊 Evaluación BiGRU"))
train_m_bigru, test_m_bigru, cm_bigru, overfit_bigru = eval_keras_model(
    bigru_model, X_train_pad, y_train, X_test_pad, y_test
)

display(Markdown(f"""
**BiGRU — Test**
- Accuracy: `{test_m_bigru['accuracy']:.3f}`
- Precision: `{test_m_bigru['precision']:.3f}`
- Recall: `{test_m_bigru['recall']:.3f}`
- F1: `{test_m_bigru['f1']:.3f}`
- ROC-AUC: `{test_m_bigru['roc_auc']:.3f}`
"""))


### 9. Selección del mejor modelo neuronal


In [ ]:
# === 9. Select best NN model =================================================

candidates_nn = [
    ("gru", gru_model, train_m_gru, test_m_gru, cm_gru),
    ("bigru", bigru_model, train_m_bigru, test_m_bigru, cm_bigru),
]

candidates_nn_sorted = sorted(
    candidates_nn,
    key=lambda x: (x[3].get("f1", 0), x[3].get("roc_auc", 0)),
    reverse=True
)

best_tag, best_nn_model, best_train_m, best_test_m, best_cm = candidates_nn_sorted[0]

final_model_name = f"{model_name_base}_{best_tag}"

display(Markdown(f"""
**🥇 Mejor modelo neuronal:** `{final_model_name}`

- F1 (test): `{best_test_m['f1']:.3f}`
- ROC-AUC (test): `{best_test_m['roc_auc']:.3f}`
"""))


### 10. Guardado del modelo neuronal y JSON

En esta celda:

- Guardamos el modelo en formato Keras:
  - `backend/models/<model_name>.keras`
- Guardamos el JSON estándar en:
  - `data/results/<model_name>.json`

Esto permite comparación directa con modelos clásicos.


In [ ]:
# === 10. Save best NN model and JSON ========================================

n_samples = df.shape[0]

results_dict = build_results_json_nn(
    model_name=final_model_name,
    target_label=TARGET_COL,
    n_samples=n_samples,
    vocab_size=vocab_size,
    train_size=len(X_train) / n_samples,
    test_size=len(X_test) / n_samples,
    random_state=RANDOM_STATE,
    test_metrics=best_test_m,
    cm_dict=best_cm,
    notes="GRU vs BiGRU; text_classic; tokenizer + embedding"
)

json_path = save_results_json(results_dict, results_dir, final_model_name)

# Save Keras model
keras_path = models_dir / f"{final_model_name}.keras"
best_nn_model.save(keras_path)

display(Markdown(f"""
**💾 Guardado completado**

- Modelo neuronal: `{keras_path}`
- JSON: `{json_path}`

Este JSON es compatible con `comparison_models.ipynb`.
"""))
